In [3]:
import os
import pandas as pd
import numpy as np

# %run 'utils.ipynb'
%run 'read_and_process_data.ipynb'

In [4]:
# build a random forest
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
clf = RandomForestClassifier()
clf

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [5]:
# train on the training set

# fit the model
clf = clf.fit(X, y)

# predict train set
train_yhat = clf.predict(X)
accuracy_score(y, train_yhat)

/usr/lib/python2.7/site-packages/ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.


0.8567415730337079

In [6]:
# predict the dev set - to see what the variance
dev_yhat = clf.predict(dev_X)
accuracy_score(dev_y, dev_yhat)

0.83240223463687146

In [7]:
# build a decision tree so we can do visualization
from sklearn.tree import DecisionTreeClassifier
tree_clf = DecisionTreeClassifier()
tree_clf = tree_clf.fit(X, y)
tree_clf

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [8]:
# visualize the tree before hyperparameter tuning

from sklearn import tree
with open('titanic.dot', 'w') as f:
    f = tree.export_graphviz(tree_clf, feature_names=features, out_file=f)

# import sys
# sys.path

In [9]:
# hyper parameter tuning
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

# import sys
# print(sys.path)

# {'max_depth': 18.0, 'max_leaf_nodes': 75.0, 'n_estimators': 20.0}
# {'max_depth': 25.0, 'max_leaf_nodes': 90.0, 'min_samples_leaf': 3.0, 'min_samples_split': 9.0, 'n_estimators': 15.0}
# {'max_depth': 27.0, 'max_leaf_nodes': 96.0, 'min_samples_leaf': 4.0, 'min_samples_split': 10.0, 'n_estimators': 19.0}

space = {
    'max_depth': hp.quniform('max_depth', 25, 30, 1),
    'max_leaf_nodes': hp.quniform('max_leaf_nodes', 90, 100, 3),
    'n_estimators': hp.quniform('n_estimators', 15, 20, 1),
    'min_samples_split': hp.quniform('min_samples_split', 8, 12, 2),
    'min_samples_leaf': hp.quniform('min_samples_leaf', 3, 9, 2),
}

def score(params):
    # convert all the numbers to integer
#     params['n_estimators'] = int(params['n_estimators'])
    print(params)
    params = {x: int(y) for x, y in params.iteritems()}
    if params['min_samples_leaf'] == 0:
        params['min_samples_leaf'] = 1
    params['n_jobs'] = -1
    clf = RandomForestClassifier(**params)
    return {'loss': 1-check_accuracy(clf, X, y, dev_X, dev_y), 'status': STATUS_OK}

trials = Trials()
print('before best')
# best = fmin(score, space, algo=tpe.suggest, trials=trials, max_evals=500)
print('after best')

best = {'max_depth': 27.0, 'max_leaf_nodes': 96.0, 'min_samples_leaf': 4.0, 'min_samples_split': 10.0, 'n_estimators': 19.0}
best

before best
{'min_samples_split': 12.0, 'max_leaf_nodes': 96.0, 'n_estimators': 18.0, 'max_depth': 28.0, 'min_samples_leaf': 6.0}


/usr/lib/python2.7/site-packages/ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """


{'min_samples_split': 12.0, 'max_leaf_nodes': 93.0, 'n_estimators': 19.0, 'max_depth': 28.0, 'min_samples_leaf': 4.0}
{'min_samples_split': 12.0, 'max_leaf_nodes': 96.0, 'n_estimators': 15.0, 'max_depth': 26.0, 'min_samples_leaf': 4.0}
{'min_samples_split': 8.0, 'max_leaf_nodes': 96.0, 'n_estimators': 16.0, 'max_depth': 27.0, 'min_samples_leaf': 6.0}
{'min_samples_split': 8.0, 'max_leaf_nodes': 99.0, 'n_estimators': 19.0, 'max_depth': 28.0, 'min_samples_leaf': 6.0}
{'min_samples_split': 8.0, 'max_leaf_nodes': 96.0, 'n_estimators': 17.0, 'max_depth': 28.0, 'min_samples_leaf': 6.0}
{'min_samples_split': 12.0, 'max_leaf_nodes': 99.0, 'n_estimators': 18.0, 'max_depth': 28.0, 'min_samples_leaf': 8.0}
{'min_samples_split': 12.0, 'max_leaf_nodes': 90.0, 'n_estimators': 18.0, 'max_depth': 25.0, 'min_samples_leaf': 6.0}
{'min_samples_split': 8.0, 'max_leaf_nodes': 96.0, 'n_estimators': 19.0, 'max_depth': 26.0, 'min_samples_leaf': 8.0}
{'min_samples_split': 10.0, 'max_leaf_nodes': 96.0, 'n_estim

KeyboardInterrupt: 

In [ ]:
# use the tune parameter on training set
best = {x: int(y) for x, y in best.iteritems()}
clf_tuned = RandomForestClassifier(**best)
clf_tuned = clf_tuned.fit(X, y)
train_pred = clf_tuned.predict(X)
accuracy_score(y, train_pred)

In [ ]:
# evaluate the tuned hyperparameter with dev set
dev_pred = clf_tuned.predict(dev_X)
accuracy_score(dev_y, dev_pred)

In [ ]:
# write prediction to file
# pred = clf_tuned.predict(test_X)
pred = clf.predict(test_X)

outfile = os.path.join(data_dir, 'prediction.csv')
df = pd.DataFrame()
df['PassengerId'] = passenger_id
df['Survived'] = pred
df.to_csv(outfile, index=False)